In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from mylibs import *

In [43]:
def get_data(url:str) -> pd.DataFrame:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    product_container = soup.find(name = 'div', attrs =  {'class' :'block-card-cate container-card-cate d-flex justify-content-flex-start flex-wrap w-10'})
    
    list_product_names = []
    tag_product_names = product_container.find_all(name= 'h3')
    for tag_product_name in tag_product_names:
        product_name = tag_product_name.text
        list_product_names.append(product_name)
        
    list_product_prices = []
    tag_product_prices = product_container.find_all(name= 'div', attrs = {'class' : 'price-card primary-price'})
    for tag_product_price in tag_product_prices:
        product_price = tag_product_price.text.split('đ')[0].replace('.', '').replace('                        ', '')
        list_product_prices.append(product_price)
        
    list_image_links = []
    tag_image_links = product_container.find_all(name= 'div', attrs = {'class' : 'image-card-cate'})
    for tag_image_link in tag_image_links:
        image_link = tag_image_link.find('img')['data-src']
        list_image_links.append(image_link)
        # img_name = image_link.split('/')[-1]
        # download_image(image_link, f'tivi/{img_name}')
    # secondary_list = []
    # tag_secondary_prices = product_container.find_all(name= 'div', attrs = {'class' : 'price-card secondary-price'})
    # for tag_secondary_price in tag_secondary_prices:
    #     # print(tag_secondary_price)
    #     if(tag_secondary_price is None):
    #         secondary_list.append(0)
    #         continue
    #     secondary_price = int(tag_secondary_price.find(name = 'span').text.replace(',', '').split('đ')[0].strip())
       
    #     secondary_list.append(secondary_price)

            
    # print(len(list_product_names), len(list_product_prices), len(list_image_links), len(secondary_list))
    df = pd.DataFrame({'product_name': list_product_names, 'product_price': list_product_prices, 'image_link': list_image_links})
    return df        

In [18]:
url = 'https://dienmaythiennamhoa.vn/tivi.html'
df = get_data(url)
df.shape[0]

32

In [44]:
url = 'https://dienmaythiennamhoa.vn/tivi.html?page=14&_is_desktop=1&_is_robot=0&sort='
page_response = requests.get(url)
i = 1
df_og = pd.DataFrame()
while page_response.status_code == 200:
    if i > 1:
        break
    df = get_data(url)
    if df.shape[0] == 0: 
        break
    i += 1
    df_og = pd.concat([df_og, df], ignore_index = True)
    page_url = url + '?page=' + str(i) + '&_is_desktop=1&_is_robot=0&sort='
    page_response = requests.get(url)
    
    
df_og.shape


(18, 3)

In [20]:
df_og.to_csv('tivi.csv', index = False)